In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

import os
from tqdm import tqdm_notebook

In [138]:
def merge_df(df):
    # 데이터 merge
    df = total_df[0].copy()
    df.fillna(0,inplace=True)
    df.replace(0,'',inplace=True)
    df['merge'] = df['title'] + df['keyword'] + df['cat1'] #+ df['cat2'] + df['cat3']
    
    # 작업용 df로 변경
    df = df[['product_id','merge']]
    return df


def preprocessing(review, okt, remove_stopwords = False, stop_words = [], test = False):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if test:
        print(review_text)
        print(word_review)

    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]

    return word_review

## stop word
stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']


# 리스트 재정리
def re_listing(info):
    total=[]
    for text in range(len(info)):
        temp = []
        for word in range(len(info[text])):
            if word == 0 :
                temp = info[text][word] 
            elif word < (len(info[text])):
                temp = temp  + ' ' + info[text][word] + ' ' 
            else:
                temp = temp + info[text][word]
        total.append(temp)
    return total

# 작업파일 불러오기

- 전체 :  ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시', '갤럭시케이스',
          '안경','모자','하이힐','후드','아이폰케이스','아이폰','자켓','청바지','점퍼','중고차','키보드','노트북','마우스','목도리',
          '네일','니트','목걸이','나이키','닌텐도','원피스','패딩','바지','향수','냉장고','레고','반지','셔츠','치마','스피커','정장',
          '선글라스','티셔츠','트레이닝','티비','지갑','세탁기','시계']
- '아디다스 0','에어컨 1','에어팟 2','오토바이 3','블라우스 4','팔찌 5','카메라 6','코트7','컴퓨터 8','구두 9','귀걸이10','갤럭시 11'

In [3]:
#내 작업
query_list = ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시']

In [4]:
total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/info/{query_list[i]}.csv'))

# 태그 추천 테스트
1. 문장-단어 TF-IDF 구하기 : 한 문장(product)에 중요도가 높은 단어들 테이블
2. 문장-문장 유사도 구하기 : 특정 문장(prodcut)기준 문장간 유사도를 통해 특정 문장과 유사한 문장 뽑기
3. 1을 미리 구해놓음 -> 2에서 비슷한 문장 뽑음 -> 2에서 뽑힌 문장을 1에서 찾음 -> 1에서 태그 뽑기


- 구현에 필요한 내용
 - 1,2번 구현 테스트
 - 언어 전처리 필요

In [105]:
adidas = merge_df(adidas)
adidas

,product_id,merge
0,162427299,"아디다스 문디알 풋살화265m아디다스,문디알,코파문디알,아디다스문디알,풋살화스포츠/레저"
1,172688578,"아디다스 반팔티아디다스,아디다스반팔,반팔남성의류"
2,174560842,아디다스 봄가을 트레이닝 1230100남성의류
3,171688500,"아디다스 아이바 글러브 레드 복싱,복싱글러브,아이바스포츠/레저"
4,174560837,"아디다스 빅로고 버건디 후드티아디다스빅로고,아디다스후드티,빅로고후드,버건디후드여성의류"
...,...,...
5995,170889615,"아디다스 x 고스티드+ 265 FG 최상급 축구화축구화,축구,아디다스x고스티드+,x..."
5996,118288417,"아디다스 유로파 핑크 하의 85아디다스 유로파,유로파 85,유로파 핑크여성의류"
5997,118288210,"아디다스 유로파 핑크 상의 85아디다스 유로파,85,유로파 핑크,아디다스 여자 상여성의류"
5998,174460778,"<240>아디다스 오즈위고 스웨이드 화이트 새상품아디다스,오즈위고,240,EG054..."


## 불용어 처리

In [129]:
from konlpy.tag import Okt
import re

In [130]:
okt = Okt()
info=[]
for word in tqdm_notebook(adidas['merge']):
    info.append(preprocessing(word, okt, remove_stopwords = True, stop_words=stop_words))

/var/folders/c0/q_ltjrb55gzc5wvhsv7s_vsh0000gn/T/ipykernel_870/4199161003.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for word in tqdm_notebook(adidas['merge']):


  0%|          | 0/6000 [00:00<?, ?it/s]

### 불용어 처리한 단어들 list 재정의

In [142]:
info = re_listing(info)
info

['아디다스 문디알  풋살  화  아디다스  문디알  코파  문디알  아디다스  문디알  풋살  화  스포츠  레저 ',
 '아디다스 반  팔  티  아디다스  아디다스  반  팔  반  팔  남성  의류 ',
 '아디다스 봄  가을  트레이닝  남성  의류 ',
 '아디다스 아이  바  글러브  레드  복싱  복싱  글러브  이바  스포츠  레저 ',
 '아디다스 빅로  고  버건디  후드  티  아디다스  빅  로고  아디다스  후드  티빅  로고  후드  버건디  후드  여성  의류 ',
 '아디다스 쪼리  아디다스  아디다스  쪼리  슬리퍼  쪼리  아디다스  슬리퍼  신발 ',
 '아디다스 니  짜다  운동화  아디다스  니  짜다  아디다스  스니커즈  운동화  아디다스  운동화  신발 ',
 '아디다스 스니커즈  아디다스  신발 ',
 '아디다스 크다  틀벨  아디다스  크다  틀벨  스포츠  레저 ',
 '라 프시  몬스  아디다스  테  렉스  판매  하다  프  시몬  스런  닝  화  라  프시  몬스  운동화  라  프시  몬스  라  프시  몬스  라  프시  몬스  신발  신발 ',
 '아디다스 신발  아디다스  사이즈  신발 ',
 '아디다스 후  리스  팬츠  아디다스  팬츠  후  리스  남성  의류 ',
 '아디다스 운동화  아디다스  운동화  아디다스  운동화  신발 ',
 '아디다스 스탠  스미스  아디다스  스탠  스미스  아디다스  신발 ',
 '팔라스 아디다스  트랙  팬츠  바지  판매  아디다스  파이어버드  트랙  트레이닝  팬츠  남성  의류 ',
 '아디다스 유로파  남빨  져지  윤두준  져지  나이키  뉴발  윤두준  져지  윤두준  데  상  트이다  성  의류 ',
 '아디다스 키즈  용  반바지  아디다스  반바지  키즈  용  스포츠  유아  동  출산 ',
 '아디다스 노바  윈드브레이커  아디다스  바람막이  윈드브레이커  남성  의류 ',
 '아디다스 깔깔이  패딩  네이비  점퍼  자켓  블루

### product별 tf-idf 구하기

In [143]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
countvect = vect.fit_transform(info)

countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = adidas['product_id']
countvect_df

,ㄷㅅ,ㅇㅓㅅㅎㄴ,ㅠㅠ,가격,가격파괴,가공,가구,가능,가다,가드,가디,가디건,가바,가방,가볍다,가보,가보다,가사,가스,가슴,가시,가오리,가을,가전,가정,가젤,가죽,가토,가트,각종,간식,간절,간지,간지나다,갈색,갈이,감다,강남,강동원,강렬하다,강민호,강아지,강추,같다,개다,개봉,개월,갸쿠,거래,거런,거리,거슬리다,거여,거위,거유,거의,거저,건담,건빵,걸즈,검검,검다,검베,검빨,검솔,검은색,검정,검정색,검파,검핑,겉옷,게스,게이츠,게임,겐조,겨울,겨울스포츠,격인,격투,경기,경량,경북,경찰차,경희대,경희대학교,계시,고급,고급스럽다,고니,고베,고샤,고스,고스로리,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,...,핫딜,핫팬츠,핫핑크,항공,해골,해링,해보다,해쉬,해외,해찬,핸더,핸더스킴부룸,핸더스킴핸,핸드,햇빛,햡니,향수,허니,허다,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬스저지파름,헬스화헬스,헬싱키,현대,현아,형광,호날두,호남성,호랑이,호사,호스,호카,호텔,호피,호호,홀로그램,홀리,홀리스,홍보,화구,화단,화보,화성인,화소,화어,화언,화이,화이트,화통,화파,화하다,환불,환영,활동,황금,회색,회실착,회착,회핑,후두,후드,후디,후디드,후디후,후루룩,후리,후망,후베이,후유,후집,휠라,휴먼,흑금,흡수,희귀,희다,흰색,흰핑,히로,히어로,히트,히피,힐탑,힙색,힙색가방힙,힙색파름,힙색힙,힙합
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
162427299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
172688578,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
174560842,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
171688500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
174560837,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170889615,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features = 10000)
tfvect = vect.fit(info)

tfidv_df = pd.DataFrame(tfvect.transform(info).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = adidas['product_id']
tfidv_df

,ㄷㅅ,ㅇㅓㅅㅎㄴ,ㅠㅠ,가격,가격파괴,가공,가구,가능,가다,가드,가디,가디건,가바,가방,가볍다,가보,가보다,가사,가스,가슴,가시,가오리,가을,가전,가정,가젤,가죽,가토,가트,각종,간식,간절,간지,간지나다,갈색,갈이,감다,강남,강동원,강렬하다,강민호,강아지,강추,같다,개다,개봉,개월,갸쿠,거래,거런,거리,거슬리다,거여,거위,거유,거의,거저,건담,건빵,걸즈,검검,검다,검베,검빨,검솔,검은색,검정,검정색,검파,검핑,겉옷,게스,게이츠,게임,겐조,겨울,겨울스포츠,격인,격투,경기,경량,경북,경찰차,경희대,경희대학교,계시,고급,고급스럽다,고니,고베,고샤,고스,고스로리,고어텍스,곤잘레스,골드,골든,골스,골키퍼,골프,...,핫딜,핫팬츠,핫핑크,항공,해골,해링,해보다,해쉬,해외,해찬,핸더,핸더스킴부룸,핸더스킴핸,핸드,햇빛,햡니,향수,허니,허다,허리,허벅지,허프,헐값,헤드,헤리티지,헤링턴,헤브,헤비,헤어,헨리,헬스,헬스장,헬스저지파름,헬스화헬스,헬싱키,현대,현아,형광,호날두,호남성,호랑이,호사,호스,호카,호텔,호피,호호,홀로그램,홀리,홀리스,홍보,화구,화단,화보,화성인,화소,화어,화언,화이,화이트,화통,화파,화하다,환불,환영,활동,황금,회색,회실착,회착,회핑,후두,후드,후디,후디드,후디후,후루룩,후리,후망,후베이,후유,후집,휠라,휴먼,흑금,흡수,희귀,희다,흰색,흰핑,히로,히어로,히트,히피,힐탑,힙색,힙색가방힙,힙색파름,힙색힙,힙합
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
162427299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172688578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174560842,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.851431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171688500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### 비슷한 prodcut 추출

In [154]:
cosine_matrix = cosine_similarity(tfidv_df,tfidv_df)
cosine_matrix.shape

(6000, 6000)

In [155]:
product_df = pd.DataFrame(cosine_matrix, columns = adidas['product_id'])
product_df.index = adidas['product_id']
product_df

product_id,162427299,172688578,174560842,171688500,174560837,165210049,165209401,174503200,168706995,96522408,148900600,148545874,169624243,162353657,164924747,101961855,163267230,174458599,115716172,114303122,110151536,40437378,170040833,174560659,169948878,172856787,167062286,163809023,169901223,169900628,169513126,174560567,123447019,170255787,173624611,169405830,169055805,171413458,173963412,174560475,169366444,159154111,174560437,153547825,153547403,172503451,159717559,159717219,93919320,161606489,146485977,172664156,173964939,108169361,131068321,105354790,131815872,114719878,139921146,159152810,105000620,168405732,174560265,172713281,173722377,137586113,153218467,138226401,104868902,174490533,115155740,165128378,166175083,166502297,142976326,166558553,174067762,173582317,148249570,104325072,109754136,167395795,173348895,173525239,174266516,171665750,102639685,171808926,170501858,168952990,170979891,171274035,167527187,156826838,172899527,174256438,172914692,171234844,104426631,174559991,...,174512463,174512361,165008442,174512337,163607296,133090957,173853025,172707123,159473164,174358096,174512244,152633221,137482827,174170147,117421202,172808371,172507347,174512137,172086681,172083008,154508788,174512075,173450213,174512037,174511993,174511981,170004776,174511936,170175734,141644085,174511914,135110750,174511880,168537916,137077322,174511828,174511824,158170737,155098350,174511751,174336110,135803157,158351242,174511674,173685577,130087387,174291547,94632138,174511592,171474757,174511530,174238322,170234713,174239098,174511510,174238484,174511473,128406476,173294620,144875341,171609494,152384603,162344224,164626710,67757102,163758209,169630355,168598891,157207127,143097613,171413979,173313264,170870407,174071149,157389335,64591767,170872002,163269126,173942393,134269768,164075974,173449453,86099225,160057106,118944463,174511139,170042233,95567456,173670643,174064124,165532282,125302868,174001613,173879632,171935287,170889615,118288417,118288210,174460778,171198937
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
162427299,1.000000,0.067857,0.010189,0.025040,0.010109,0.013027,0.019642,0.032218,0.034092,0.001788,0.028542,0.013422,0.024033,0.017594,0.011040,0.002748,0.028417,0.011105,0.005188,0.009156,0.003523,0.005036,0.007626,0.011968,0.006203,0.017836,0.012327,0.017152,0.014111,0.009613,0.012807,0.006682,0.014910,0.014361,0.021847,0.023568,0.011627,0.011661,0.007924,0.007551,0.006694,0.003306,0.027679,0.011787,0.014024,0.008802,0.006849,0.009338,0.010501,0.015746,0.008686,0.005691,0.003258,0.005508,0.007535,0.010695,0.004459,0.006862,0.020937,0.012562,0.010535,0.030841,0.004640,0.016455,0.009874,0.004366,0.004973,0.016282,0.004223,0.009352,0.025950,0.079293,0.000000,0.014128,0.016058,0.005108,0.011709,0.011159,0.011762,0.014254,0.009417,0.020865,0.020720,0.057113,0.024542,0.023147,0.017315,0.012510,0.023461,0.013029,0.020992,0.004910,0.051654,0.002560,0.026727,0.012865,0.033739,0.079227,0.090230,0.008628,...,0.004801,0.077272,0.011804,0.032358,0.013893,0.009275,0.003876,0.018380,0.019752,0.054039,0.005484,0.000000,0.004759,0.004187,0.014716,0.023615,0.012461,0.035231,0.021473,0.044470,0.020871,0.021882,0.022550,0.011138,0.036124,0.013617,0.018106,0.010330,0.018677,0.012253,0.036124,0.009505,0.012472,0.006710,0.005766,0.025696,0.009602,0.006382,0.018643,0.034844,0.005251,0.005890,0.055113,0.054843,0.011912,0.012413,0.002772,0.004653,0.012610,0.004890,0.019008,0.010564,0.003701,0.037225,0.012066,0.006582,0.032358,0.009215,0.003346,0.035504,0.012399,0.051368,0.012320,0.009255,0.033176,0.071151,0.038895,0.010795,0.021669,0.005605,0.017196,0.026516,0.020718,0.016714,0.017409,0.021002,0.018840,0.023607,0.056378,0.023222,0.020550,0.012832,0.010766,0.005612,0.008517,0.067857,0.023629,0.004976,0.019157,0.019756,0.012180,0.005055,0.045680,0.018639,0.038351,0.032772,0.0

### top3 상품별 중요도 높은 키워드 top5 고르기

In [156]:
# 특정 상품지정 및 유사한 product 3개 뽑기
top3 = product_df.iloc[2].sort_values(ascending=False)[1:4].index

# product_id str로 변환
product_df.index = product_df.index.astype(str)
tfidv_df.index = tfidv_df.index.astype(str)

# top3 상품의 연관성 높은 키워드 top5 뽑기
total_tag =[]
for i in range(len(top3)):
    tfidv_df[tfidv_df.index == str(top3[i])]
    temp =[]
    # 유사 상품중 top5 키워드 추천
    temp.append(tfidv_df[tfidv_df.index == str(top3[i])].T.sort_values(by = str(top3[i]), ascending=False)[:5].index.to_list())

    # top5 중복 제거
    for i in range(len(temp[0])):
        total_tag.append(temp[0][i])
total_tag = list(set(total_tag))
total_tag

['트레이닝', '의류', '남성', '익스', '팬츠', '아디다스']

In [157]:
top3

Int64Index([104662468, 143268082, 144153808], dtype='int64', name='product_id')

# 방향성 재정의
- 제목 + 카테고리 = feature
- 태그 = label
- feature